# imports

In [1]:
import pickle
from srgnn_pl import SRGNN_model, SRGNN_Map_Dataset, SRGNN_sampler
from utils import fake_parser
import torch
import os

from torch.utils.data import DataLoader
import pytorch_lightning as pl

import numpy as np

In [2]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import yaml
from math import ceil
from sklearn.mixture import GaussianMixture
from IPython.display import clear_output

# data loading

In [3]:
torch.set_float32_matmul_precision('medium')

In [5]:
global_run_id='run-20240209_162656-h23ej73g'

In [6]:
with open(f"./wandb/{global_run_id}/files/config.yaml", "r") as stream:
        config=yaml.safe_load(stream)

keys=list(config.keys())
for k in keys:
    if k not in fake_parser().__dict__.keys():
        del config[k]
    else:
        config[k]=config[k]['value']

opt=fake_parser(**config)
print(opt.__dict__)

model=SRGNN_model.load_from_checkpoint(f"./GNN_master/{global_run_id.split('-')[-1]}/checkpoints/"+
                                       os.listdir(f"./GNN_master/{global_run_id.split('-')[-1]}/checkpoints/")[0], opt=opt)

{'dataset': 'yoochoose_custom', 'batchSize': 128, 'hiddenSize': 64, 'epoch': 60, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 3, 'patience': 10, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.2, 'pretrained_embedings': True, 'unfreeze_epoch': 1}


In [7]:
with open(f'./gmm_32_kmeans_{opt.hiddenSize}.gmm', 'rb') as gmm_file:
    gm=pickle.load(gmm_file)
#    session_labels=gm.predict(session_emb)

In [8]:
test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))
test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)
test_dataloader=DataLoader(test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(test_dataset, opt.batchSize, shuffle=False, drop_last=False),
                             drop_last=False
                            )

data masking start
data masking 1
data masking 2
data masking 3
done masking


# global model results

# evaluate each cluster model

## all sessions

In [9]:
session_emb=[]

model.to('cuda')
for batch in tqdm(test_dataloader, total=test_dataset.length//opt.batchSize):
    batch=[b.to('cuda') for b in batch]
    session_emb.append(model.get_session_embeddings(batch).cpu().detach().numpy())
model.to('cpu')
session_emb=np.concatenate(session_emb)

1214it [00:06, 176.86it/s]                          


In [10]:
session_labels=[]
for i in tqdm(range(ceil(session_emb.shape[0]/opt.batchSize))):
    session_labels.append(gm.predict(session_emb[i*opt.batchSize: (i+1)*opt.batchSize]))
session_labels=np.concatenate(session_labels)

100%|██████████| 1214/1214 [00:01<00:00, 841.14it/s]


In [12]:
hit,mrr=[],[]
model.to('cuda')
for batch in tqdm(test_dataloader, total=test_dataset.length//opt.batchSize):
    batch=[b.to('cuda') for b in batch]
    sub_scores, targets=model.predict_step(batch)
    targets=targets.flatten()
    for score, target in zip(sub_scores, targets):
        correct_pred=torch.isin(target - 1, score).cpu()
        hit.append(correct_pred)
        if not correct_pred:
            mrr.append(0)
        else:
            mrr.append(1 / (torch.where(score == target - 1)[0][0] + 1).cpu())
model.to('cpu')
hit=np.array(hit)
mrr=np.array(mrr)

1214it [00:44, 27.11it/s]                          


### finetuned

In [ ]:
runs_df=pd.read_csv('./wandb_export_gm_finetune_runs.csv')
runs_df['cluster']=runs_df.Name.map(lambda x: int(x.split('_')[-1]))

### trained from scratch

In [ ]:
runs_df=pd.read_csv('./wandb_export_gm_all_runs.csv')
runs_df['cluster']=runs_df.Name.map(lambda x: int(x.split('_')[-1]))

In [13]:
cluster_results=[]
cluster_results.append(
{'cluster':-1, 
 'test_loss': np.nan,
 'test_hit': np.average(hit),
 'test_mrr': np.average(mrr)})

for c in tqdm(range(gm.n_components)):
    clear_output(wait=True)
    idxs=np.arange(len(test_data[0]))[session_labels==c]

    cluster_sess=[test_data[0][i] for i in idxs]
    cluster_targets=[test_data[1][i] for i in idxs]
    cluster_data=(cluster_sess,cluster_targets)
    cluster_dataset=SRGNN_Map_Dataset(cluster_data, shuffle=False)
    cluster_dataloader=DataLoader(cluster_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(cluster_dataset, opt.batchSize, shuffle=False, drop_last=False),
                             drop_last=False
                            )
    run_id=runs_df.loc[runs_df.cluster==c].ID.item()
    cluster_model=SRGNN_model.load_from_checkpoint(f"./GNN_master/{run_id}/checkpoints/"+
                                       os.listdir(f"./GNN_master/{run_id}/checkpoints/")[0], opt=opt)
    
    trainer=pl.Trainer(limit_test_batches=ceil(cluster_dataset.length/opt.batchSize),limit_predict_batches=ceil(cluster_dataset.length/opt.batchSize))
    metrics=trainer.test(cluster_model, cluster_dataloader)[0]#only one dataloader
    metrics['cluster']=c
    metrics['global_model_hit']=100*np.average(hit[idxs])
    metrics['global_model_mrr']=100*np.average(mrr[idxs])
    cluster_results.append(metrics)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


data masking start
data masking 1
data masking 2
data masking 3
done masking


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_hit          │    0.36204269528388977    │
│         test_loss         │    11.874570846557617     │
│         test_mrr          │    0.04756525531411171    │
└───────────────────────────┴───────────────────────────┘

100%|██████████| 32/32 [01:11<00:00,  2.22s/it]


In [14]:
cluster_results=pd.DataFrame(cluster_results)

In [15]:
cluster_results

,cluster,test_loss,test_hit,test_mrr,global_model_hit,global_model_mrr
0,-1,NaN,0.553531,0.248305,NaN,NaN
1,0,26.611471,0.831330,0.148933,55.021351,24.667072
2,1,10.883867,2.296442,0.890496,55.911824,32.335965
3,2,9.350189,7.946135,2.162092,54.569892,22.735172
4,3,10.921228,0.789494,0.171856,24.083590,9.477628
5,4,7.368024,36.409153,15.105134,50.296424,20.445324
6,5,11.659607,1.852679,0.428738,63.672922,27.894669
7,6,9.130525,4.165573,1.106665,62.656582,26.534482
8,7,10.206665,2.900764,0.830030,52.854974,26.665005
9,8,11.461602,0.746528,0.258960,59.394572,27.624812


In [80]:
(cluster_results.test_hit>cluster_results.global_model_hit).any()

False

## full sessions

In [124]:
full_session_emb=[]
full_sessions=[]
for batch in tqdm(test_dataloader, total=test_dataset.length//opt.batchSize):
    # use only original sessions
    a=batch[3][0]
    a=torch.vstack([torch.zeros(a.shape[1]), a])
    idxs=torch.diff(a, dim=0).sum(axis=1)
    for i in range(len(batch)):
        batch[i]=batch[i][:,idxs>0]
    #print(batch[0].shape)
    #break
    full_sessions.append(batch)
    full_session_emb.append(model.get_session_embeddings(batch).detach().numpy())
full_session_emb=np.concatenate(full_session_emb)

152it [00:13, 11.67it/s]                         


In [125]:
full_session_labels=gm.predict(full_session_emb)

In [126]:
len(full_session_labels)

34034

In [127]:
hit,mrr=[],[]
for batch in tqdm(full_sessions):
    sub_scores, targets=model.predict_step(batch)
    targets=targets.flatten()
    for score, target in zip(sub_scores, targets):
        correct_pred=torch.isin(target - 1, score)
        hit.append(correct_pred)
        if not correct_pred:
            mrr.append(0)
        else:
            mrr.append(1 / (torch.where(score == target - 1)[0][0] + 1))
hit=np.array(hit)
mrr=np.array(mrr)

100%|██████████| 152/152 [00:12<00:00, 12.41it/s]


In [129]:
full_sess_results=[]
full_sess_results.append(
{'cluster':-1, 
 'global_model_hit': np.average(hit),
 'global_model_mrr': np.average(mrr)})

for c in tqdm(range(gm.n_components)):
    clear_output(wait=True)
    idxs=full_session_labels==c

    run_id=runs_df.loc[runs_df.cluster==c].ID.item()
    cluster_model=SRGNN_model.load_from_checkpoint(f"./GNN_master/{run_id}/checkpoints/"+
                                       os.listdir(f"./GNN_master/{run_id}/checkpoints/")[0], opt=opt)
    cluster_model.to('cpu')
    
    chit,cmrr=[],[]
    counter=0
    prev_batch=None
    for batch in (full_sessions):
        batch_idxs=idxs[counter:counter+batch[4].shape[-1]]
        counter+=batch[4].shape[-1]

        batch=[b[:, batch_idxs] for b in batch]
        

        sub_scores, targets=cluster_model.predict_step(batch)
        targets=targets.flatten()
        for score, target in zip(sub_scores, targets):
            correct_pred=torch.isin(target - 1, score)
            chit.append(correct_pred)
            if not correct_pred:
                cmrr.append(0)
            else:
                cmrr.append(1 / (torch.where(score == target - 1)[0][0] + 1))

    metrics={}
    metrics['cluster']=c
    metrics['model_hit']=100*np.average(chit)
    metrics['model_mrr']=100*np.average(cmrr)
    metrics['global_model_hit']=100*np.average(hit[idxs])
    metrics['global_model_mrr']=100*np.average(mrr[idxs])
    full_sess_results.append(metrics)

100%|██████████| 32/32 [00:16<00:00,  1.90it/s]


In [131]:
full_sess_results=pd.DataFrame(full_sess_results)

In [136]:
full_sess_results.iloc[(full_sess_results.model_mrr>full_sess_results.global_model_mrr).values]

,cluster,test_loss,test_hit,test_mrr,model_hit,model_mrr,global_model_hit,global_model_mrr
10,9,NaN,NaN,NaN,61.501597,28.099945,62.616822,27.867965
15,14,NaN,NaN,NaN,68.902439,35.281173,68.471954,34.602202


In [135]:
full_sess_results.iloc[(full_sess_results.model_hit>full_sess_results.global_model_hit).values]

,cluster,test_loss,test_hit,test_mrr,model_hit,model_mrr,global_model_hit,global_model_mrr
15,14,NaN,NaN,NaN,68.902439,35.281173,68.471954,34.602202
26,25,NaN,NaN,NaN,63.004172,24.414224,62.054795,24.630326
